In [1]:
import pymysql
import pandas as pd
from datetime import date, timedelta

In [2]:
connection = pymysql.connect(host = '119.74.24.181', user = 'htx', password = 'Police123456', database = 'ASTRO')

In [3]:
df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection)


C:\Users\clair\AppData\Local\Temp\ipykernel_4956\4218794030.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection)


In [171]:
df_division = df[['division_assigned']].copy()
df_division.dropna(subset = ['division_assigned'], inplace = True)

df_division = df_division[df_division.division_assigned != '-']
df_division = df_division.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

values_to_be_replaced = ['CAD', 'ASCOM', 'CID']
for value in values_to_be_replaced:
    df_division = df_division.replace('.*{}.*'.format(value), value, regex=True)
 
df_division = df_division.replace(['ASCOM', 'ASD', 'CID', 'TCIB', 'UMSF', 'ICB', 'APD', 'ACB', 'GOIS'], 'A')
df_division = df_division.groupby('division_assigned').size().reset_index(name = 'No_of_cases_per_division')

df_division = df_division.drop(index = [0,4,5,7,9,12,13,15,16,17,18,19])

df_division = df_division.sort_values('No_of_cases_per_division', ascending = False)
df_division = df_division.reset_index(drop = True)

df_latlong = pd.DataFrame({'division_assigned': ['F', 'G', 'J', 'A', 'L', 'E', 'D', 'CAD'],
                           'latitude': ['1.3849575528196754', '1.3328211729396362', '1.3511672381459414', '1.2787649220017296', '1.4334351610154739', '1.3130793731128103', '1.3173533926882381', '1.278724700500034'],
                           'longitude': ['103.84533643093292', '103.93718201189469', '103.7023869984011', '103.8397629560721', '103.77890231189434', '103.8467156436066', '103.76671172354014', '103.8393872523757'],
                           'name': ['Ang Mo Kio Police Division HQ', 'Bedok Police Division HQ', 'Jurong Police Division Headquarters', 'Central Police Divisional Headquarters', 'Woodlands Police Division Headquarters', 'Tanglin Police Division HQ', 'Clementi Police Divisional Headquarters', 'Commercial Affairs Department']})



df_division = pd.merge(df_division, df_latlong, on = 'division_assigned')
df_division



,division_assigned,No_of_cases_per_division,latitude,longitude,name
0,F,16337,1.3849575528196754,103.84533643093292,Ang Mo Kio Police Division HQ
1,G,14133,1.3328211729396362,103.93718201189469,Bedok Police Division HQ
2,J,13487,1.3511672381459414,103.7023869984011,Jurong Police Division Headquarters
3,A,11479,1.2787649220017296,103.8397629560721,Central Police Divisional Headquarters
4,L,11021,1.4334351610154739,103.77890231189434,Woodlands Police Division Headquarters
5,E,7490,1.3130793731128103,103.8467156436066,Tanglin Police Division HQ
6,D,7226,1.3173533926882381,103.76671172354014,Clementi Police Divisional Headquarters
7,CAD,2522,1.278724700500034,103.8393872523757,Commercial Affairs Department


In [9]:
df_day = pd.Series(df['date_assigned']).value_counts().sort_index()
df_day = df_day.rename_axis('date').reset_index(name = 'no_of_cases')

df_day['date'] = pd.to_datetime(df_day['date']) - pd.to_timedelta(7, unit='d')

df_week = df_day.groupby([pd.Grouper(key='date', freq='W')])['no_of_cases'].sum().reset_index(name = 'no_of_cases_per_week')
df_week['date'] = df_week['date'].dt.strftime('%Y-%m-%d')
df_week['date'].tolist()


['2021-12-26',
 '2022-01-02',
 '2022-01-09',
 '2022-01-16',
 '2022-01-23',
 '2022-01-30',
 '2022-02-06',
 '2022-02-13',
 '2022-02-20',
 '2022-02-27',
 '2022-03-06',
 '2022-03-13',
 '2022-03-20',
 '2022-03-27',
 '2022-04-03',
 '2022-04-10',
 '2022-04-17',
 '2022-04-24',
 '2022-05-01',
 '2022-05-08',
 '2022-05-15',
 '2022-05-22',
 '2022-05-29',
 '2022-06-05',
 '2022-06-12',
 '2022-06-19',
 '2022-06-26',
 '2022-07-03',
 '2022-07-10',
 '2022-07-17',
 '2022-07-24',
 '2022-07-31',
 '2022-08-07',
 '2022-08-14',
 '2022-08-21',
 '2022-08-28',
 '2022-09-04',
 '2022-09-11',
 '2022-09-18',
 '2022-09-25',
 '2022-10-02',
 '2022-10-09',
 '2022-10-16',
 '2022-10-23',
 '2022-10-30',
 '2022-11-06',
 '2022-11-13',
 '2022-11-20',
 '2022-11-27',
 '2022-12-04',
 '2022-12-11',
 '2022-12-18',
 '2022-12-25',
 '2023-01-01',
 '2023-01-08',
 '2023-01-15',
 '2023-01-22',
 '2023-01-29',
 '2023-02-05',
 '2023-02-12',
 '2023-02-19',
 '2023-02-26',
 '2023-03-05',
 '2023-03-12',
 '2023-03-19']

In [15]:
df_test = pd.DataFrame({'date': pd.date_range(start='4/1/2023', freq='D', periods=15),
                   'sales': [6, 8, 9, 5, 4, 8, 8, 3, 5, 9, 8, 3, 4, 7, 7]})


df_test['date'] = pd.to_datetime(df_test['date']) - pd.to_timedelta(7, unit='d')
df_test.groupby([pd.Grouper(key='date', freq='W')])['sales'].sum()



date
2023-03-26    14
2023-04-02    42
2023-04-09    38
Freq: W-SUN, Name: sales, dtype: int64

In [74]:
df.drop(['initial_report_number', 'account_number', 'account_phone_number', 'whatsapp_number', 'remarks', 'account_holder_name', 'account_holder_nric', 'acount_holder_phone_number', 'account_or_paynowlah_number'], axis = 'columns', inplace=True)
print(df.shape)
print(df.info())
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

(83901, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83901 entries, 0 to 83900
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   report_number                     83901 non-null  object 
 1   classification                    83901 non-null  object 
 2   scam_type                         83901 non-null  object 
 3   account_bank                      83901 non-null  object 
 4   amount_scammed                    83900 non-null  float64
 5   amount_transcated                 5438 non-null   float64
 6   online_market_place               10034 non-null  object 
 7   division_assigned                 83732 non-null  object 
 8   date_assigned                     83901 non-null  object 
 9   telco_report_number               79106 non-null  object 
 10  overseas_local                    83900 non-null  object 
 11  asc_do                            83901 non-null  objec

In [75]:
#Number of losses
total_loss = df['amount_scammed'].sum()
print(total_loss)

625751982.2299999


In [76]:
#Victims by bank
df.groupby('bank').size().reset_index(name = 'Victims By Banks')

KeyError: 'bank'

In [ ]:
#Types of scams
df.loc[3, 'scam_type'] = 'Job Scam'
df.groupby('scam_type').size()

scam_type
E-commerce Scam     1
Job Scam           16
Loan Scam           3
Love                1
dtype: int64

In [ ]:
#Types of scams
df.loc[3, 'scam_type'] = 'Job Scam'
df.groupby('scam_type').size()